# <span style = "color:#00994c">**import**

In [5]:
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import torch
# %pip install plotly (jupyter notebook)
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "plotly_mimetype+notebook"
test_path = "./test.csv"
train_path = "./train.csv"

In [6]:
train = pd.read_csv(train_path).drop(columns = ["id"])
train_len = len(train)
test = pd.read_csv(test_path)
id_test = test["id"]
test = pd.read_csv(test_path).drop(columns = ["id"])

In [7]:
dataset = pd.concat([train,test],axis=0)
dataset = dataset.drop(columns = ["father","mother","gender"])

In [8]:
dataset

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A,NaN
171,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A,NaN
172,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G,NaN
173,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A,NaN


# <span style = "color:#00994c">**Preprocessing**

In [9]:
_t = []
for val in dataset.SNP_01 == "G G":
    if val == True:
        _t.append(1)
    else:
        _t.append(0)
dataset["has01GG"] = _t

_t = []
for val in dataset.SNP_02 == "A A":
    if val == True:
        _t.append(1)
    else:
        _t.append(0)
dataset["has02AA"] = _t

In [10]:
def create_col(dataset,col,value):
    _t = []
    for val in dataset[col] == value:
        if val == True:
            _t.append(1)
        else:
            _t.append(0)
    
    col_name_base = "has"+col[-2:]
    value_name = ""
    for chr in value:
        if chr != " ":
            value_name+=chr
    col_name = col_name_base+value_name
    print(col_name)
    dataset[col_name] = _t

    return dataset

dataset = create_col(dataset,"SNP_03","A A")
dataset = create_col(dataset,"SNP_04","G G")
dataset = create_col(dataset,"SNP_05","C C")
dataset = create_col(dataset,"SNP_06","A A")
dataset = create_col(dataset,"SNP_07","A A")
dataset = create_col(dataset,"SNP_07","G G")
dataset = create_col(dataset,"SNP_08","G G")

dataset = create_col(dataset,"SNP_09","A A")
dataset = create_col(dataset,"SNP_09","G G")
dataset = create_col(dataset,"SNP_11","A A")

dataset = create_col(dataset,"SNP_12","A A")
dataset = create_col(dataset,"SNP_12","G G")

dataset = create_col(dataset,"SNP_13","A A")
dataset = create_col(dataset,"SNP_14","A A")

has03AA
has04GG
has05CC
has06AA
has07AA
has07GG
has08GG
has09AA
has09GG
has11AA
has12AA
has12GG
has13AA
has14AA


In [11]:
#one-hot encoding for distance base algorithm
dataset_ohe = pd.get_dummies(dataset,columns = dataset.columns.drop("class"),drop_first=True) #multicollinearity를 막기위한 drop_first 옵션
train_ohe = dataset_ohe[:train_len].copy()
test_ohe = dataset_ohe[train_len:].copy().drop(columns="class")

class_map = {"A":0,"B":1,"C":2}
train_ohe["class"]=train_ohe["class"].map(class_map).astype(int)
X_train_ohe = train_ohe.drop(columns = "class")
Y_train_ohe = train_ohe["class"]

In [12]:
Y_train_ohe

0      1
1      2
2      1
3      0
4      2
      ..
257    1
258    2
259    0
260    0
261    1
Name: class, Length: 262, dtype: int32

In [13]:
import torch
import torch.nn as nn


In [14]:
X_train_ohe = torch.from_numpy(X_train_ohe.values).float()
#y_train_ohe = torch.from_numpy(pd.get_dummies(Y_train_ohe).values).float()
Y_train_ohe = torch.from_numpy(Y_train_ohe.values).long()

In [15]:
Y_train_ohe

tensor([1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 0, 0, 0, 0, 2, 1, 1, 0, 1,
        2, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0,
        2, 0, 0, 2, 0, 2, 1, 1, 0, 0, 2, 2, 1, 0, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2,
        0, 1, 1, 1, 2, 1, 2, 0, 1, 1, 2, 0, 2, 0, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1,
        2, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2,
        1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 2, 2, 1,
        1, 1, 2, 2, 0, 1, 0, 2, 2, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 0,
        0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 1, 2, 2, 1,
        1, 1, 0, 1, 1, 2, 1, 2, 0, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2,
        0, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0,
        0, 2, 2, 1, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1])

In [16]:
X_train_ohe.requires_grad = True
X_train_ohe

tensor([[1., 0., 1.,  ..., 0., 1., 1.],
        [1., 1., 0.,  ..., 0., 0., 1.],
        [1., 0., 1.,  ..., 0., 1., 1.],
        ...,
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 1.]], requires_grad=True)

# <span style = "color:#00994c">**Dimension Reduction using autoencoder with pytorch**

In [17]:
class Encoder(nn.Module):
    def __init__(self,in_features,encoding_features):
        super().__init__()
        self.in_features = in_features
        self.encoding_features = encoding_features
        self.linr = torch.nn.Linear(in_features,encoding_features)
        self.active_func = torch.nn.ReLU()
    def forward(self,x):
        out = self.active_func(self.linr(x))
        return out
        
class Decoder(nn.Module):
    def __init__(self,encoding_features,out_features):
        super().__init__()
        self.encoding_features = encoding_features
        self.out_features = out_features
        self.linr = torch.nn.Linear(encoding_features,out_features)
    def forward(self,x):
        out = self.linr(x)
        return out

class AutoEncoder(nn.Module):
    def __init__(self,in_features,encoding_features):
        super().__init__()
        out_features = in_features
        self.encoder = Encoder(in_features,encoding_features)
        self.decoder = Decoder(encoding_features,out_features)
    def forward(self,x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

## **encoding dimension=3**

### **training autoencoder**

In [25]:
autoencoder = AutoEncoder(47,3)
loss_fn = torch.nn.MSELoss()
relu = torch.nn.LeakyReLU()
optimizer = torch.optim.Adam(autoencoder.parameters(),lr=0.001)

In [26]:
for epoch in range(50000):
    #1.yhat
    out = autoencoder(X_train_ohe)
    #2
    loss = loss_fn(out,X_train_ohe)
    #3
    loss.backward()
    if epoch % 5 == True:
        print(loss.tolist())
    #4
    optimizer.step()
    optimizer.zero_grad()

0.4424034655094147
0.42862626910209656
0.4182080924510956
0.41015228629112244
0.4036443829536438
0.39814478158950806
0.39324694871902466
0.3886672854423523
0.3841835558414459
0.3797030746936798
0.37505707144737244
0.3703556954860687
0.3655255436897278
0.36067119240760803
0.35563355684280396
0.350367933511734
0.3447824716567993
0.3387194275856018
0.3321831524372101
0.32564467191696167
0.3188822567462921
0.3121398985385895
0.30546995997428894
0.2989056706428528
0.29250994324684143
0.2862774133682251
0.28021663427352905
0.2743162512779236
0.2685762643814087
0.2629985809326172
0.2575865089893341
0.2523467242717743
0.247264564037323
0.24235336482524872
0.23761920630931854
0.2330591082572937
0.22867268323898315
0.22445635497570038
0.2204071581363678
0.21652311086654663
0.21280184388160706
0.20924058556556702
0.20583564043045044
0.20258285105228424
0.19948099553585052
0.19652752578258514
0.1937192976474762
0.19105274975299835
0.188524067401886
0.18612901866436005
0.1838633120059967
0.18172226

### **visualization**

In [35]:
class_map_inv = {}
for key,value in class_map.items():
    class_map_inv[value] = key
class_map_inv

{0: 'A', 1: 'B', 2: 'C'}

In [36]:
dt_visual = pd.DataFrame({"class":Y_train_ohe})
dt_visual = pd.concat([pd.DataFrame(np.array(autoencoder.encoder(X_train_ohe).tolist())),dt_visual],axis=1)
dt_visual = dt_visual.rename(columns = {0:"x",1:"y",2:"z"})

In [37]:
count = 0
data = []
for cl in dt_visual["class"].unique():
    cond = dt_visual["class"] == cl
    _data = dt_visual.loc[cond,:]
    x = _data.x.tolist()
    y = _data.y.tolist()
    z = _data.z.tolist()
    if count == 0:
        color = "red"
    elif count == 1:
        color = "blue"
    else:
        color = "black"
    trace=go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode="markers",
        marker = dict(color = color,size=5),
        name = str(class_map_inv[cl])
        )
    data.append(trace)
    count+=1

layout = go.Layout(title=dict(text = "vectors"))

#4. figure
fig = go.Figure(data=data,layout=layout)
fig.show()

## **encoding dimension = 10**

### **training autoencoder**

In [40]:
autoencoder = AutoEncoder(47,10)
loss_fn = torch.nn.MSELoss()
relu = torch.nn.LeakyReLU()
optimizer = torch.optim.Adam(autoencoder.parameters(),lr=0.001)

In [41]:
for epoch in range(50000):
    #1.yhat
    out = autoencoder(X_train_ohe)
    #2
    loss = loss_fn(out,X_train_ohe)
    #3
    loss.backward()
    if epoch % 5 == True:
        print(loss.tolist())
    #4
    optimizer.step()
    optimizer.zero_grad()

0.41205155849456787
0.39513373374938965
0.38265979290008545
0.37270796298980713
0.3634822368621826
0.3532215654850006
0.34047237038612366
0.3254643678665161
0.30877721309661865
0.2914559841156006
0.2744297385215759
0.25841861963272095
0.2439020574092865
0.2308785617351532
0.21909144520759583
0.20829908549785614
0.19836436212062836
0.18932387232780457
0.1811644434928894
0.17377935349941254
0.16705605387687683
0.16094107925891876
0.15537405014038086
0.15034130215644836
0.14575256407260895
0.14155949652194977
0.13777706027030945
0.13436412811279297
0.1312715709209442
0.12846776843070984
0.12592263519763947
0.12359949201345444
0.12146855890750885
0.11950432509183884
0.11768952012062073
0.11601467430591583
0.11446746438741684
0.11303333193063736
0.11170299351215363
0.11046329140663147
0.10930382460355759
0.10821326076984406
0.10718654096126556
0.10621842741966248
0.10530288517475128
0.10443452000617981
0.10360894352197647
0.10282149910926819
0.102068692445755
0.10134775936603546
0.100656263